In [8]:
from google.colab import files
import pandas as pd

# Upload planet.csv file
uploaded = files.upload()

# Load the data
filename = list(uploaded.keys())[0]  # Gets your uploaded filename
print(f"Uploaded: {filename}")

Saving planet.csv to planet (1).csv
Uploaded: planet (1).csv


In [9]:
import numpy as np

# Load and clean the CSV
df = pd.read_csv(filename)
print("Original shape:", df.shape)

# Keep only numeric data, drop any corrupt rows
df = df.apply(pd.to_numeric, errors='coerce').dropna()
print("Clean shape:", df.shape)

# Extract coordinates (skip the color column)
planet_coords = df.iloc[:, 1:].values.astype(float)

# Calculate planet radius (from first point)
R = np.linalg.norm(planet_coords[0, :3])
print(f"Planet radius: {R:.2f}")

# If R is too small (<1.0), scale up the coordinates
if R < 1.0:
    planet_coords *= 10  # Makes radius 10x larger
    R = np.linalg.norm(planet_coords[0, :3])
    print(f"Adjusted radius: {R:.2f}")

Original shape: (5001, 1501)
Clean shape: (5001, 1501)
Planet radius: 0.08
Adjusted radius: 0.78


In [13]:
import numpy as np
from scipy.spatial import cKDTree
from tqdm import tqdm

# Parameters
epsilon = 0.02  # Keep this spacing
n_bugs_per_color = 100
batch_size = 1000  # Generate 1000 candidates at once

# Prepare data
existing_points = planet_coords.reshape(-1, 3)
tree = cKDTree(existing_points)
bugs = []

for color_idx in [0, 1, 2]:
    color_name = ['yellow', 'blue', 'red'][color_idx]
    print(f"\nGenerating {n_bugs_per_color} {color_name} bugs:")

    # Pre-allocate memory
    new_points = np.empty((n_bugs_per_color, 3))
    accepted = 0

    with tqdm(total=n_bugs_per_color) as pbar:
        while accepted < n_bugs_per_color:
            # Generate BATCH of random points on sphere
            theta = np.random.uniform(0, 2*np.pi, size=batch_size)
            phi = np.arccos(np.random.uniform(-1, 1, size=batch_size))
            x = R * np.sin(phi) * np.cos(theta)
            y = R * np.sin(phi) * np.sin(theta)
            z = R * np.cos(phi)
            candidates = np.column_stack((x, y, z))

            # Batch distance check
            distances, _ = tree.query(candidates, k=1)
            valid_mask = distances >= epsilon

            # Add valid points
            num_valid = min(np.sum(valid_mask), n_bugs_per_color - accepted)
            if num_valid > 0:
                new_points[accepted:accepted+num_valid] = candidates[valid_mask][:num_valid]
                accepted += num_valid
                pbar.update(num_valid)

            # Update tree periodically
            if accepted > 0 and accepted % 20 == 0:
                tree = cKDTree(np.vstack([existing_points, new_points[:accepted]]))

    # Add to final bugs list
    bugs.extend([[color_idx, *p] for p in new_points])

# Save results
bugs_array = np.array(bugs)
bugs_expanded = np.hstack([bugs_array[:, [0]], np.tile(bugs_array[:, 1:], (1, 500))])
pd.DataFrame(bugs_expanded).to_csv("bugs.csv", index=False, header=False)
print("Done! Saved to bugs.csv")


Generating 100 yellow bugs:


100%|██████████| 100/100 [00:01<00:00, 55.56it/s]



Generating 100 blue bugs:


100%|██████████| 100/100 [00:01<00:00, 55.89it/s]



Generating 100 red bugs:


100%|██████████| 100/100 [00:01<00:00, 55.23it/s]


Done! Saved to bugs.csv


In [14]:
import pandas as pd
bugs_df = pd.read_csv("bugs.csv", header=None)
print(f"Generated {len(bugs_df)} bugs")
print("First 3 rows:\n", bugs_df.head(3))

Generated 300 bugs
First 3 rows:
    0         1         2         3         4         5         6         7     \
0   0.0 -0.388812 -0.668873  0.062528 -0.388812 -0.668873  0.062528 -0.388812   
1   0.0 -0.516490 -0.293729 -0.499436 -0.516490 -0.293729 -0.499436 -0.516490   
2   0.0 -0.464009  0.602139  0.156843 -0.464009  0.602139  0.156843 -0.464009   

       8         9     ...      1491      1492      1493      1494      1495  \
0 -0.668873  0.062528  ...  0.062528 -0.388812 -0.668873  0.062528 -0.388812   
1 -0.293729 -0.499436  ... -0.499436 -0.516490 -0.293729 -0.499436 -0.516490   
2  0.602139  0.156843  ...  0.156843 -0.464009  0.602139  0.156843 -0.464009   

       1496      1497      1498      1499      1500  
0 -0.668873  0.062528 -0.388812 -0.668873  0.062528  
1 -0.293729 -0.499436 -0.516490 -0.293729 -0.499436  
2  0.602139  0.156843 -0.464009  0.602139  0.156843  

[3 rows x 1501 columns]
